In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/FontGenerator


Mounted at /content/drive
/content/drive/MyDrive/FontGenerator


In [ ]:
import zipfile
import os
import numpy as np

with zipfile.ZipFile('fonts.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/unzipped_fonts')

# Check the extracted contents
os.listdir('/content/unzipped_fonts')


# Load a specific font data from an unzipped .npy file
#font_data = np.load('unzipped_fonts/font_A.npy')  # Adjust the path and filename




['Images', 'character_fonts (with handwritten data).npz', 'character_font.npz']

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# path to dataset
dataset_path = '/content/unzipped_fonts/Images/'

# image transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((64, 64)),   # Resize
    transforms.ToTensor(),         # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1] for GAN
])

# Load the dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Create the DataLoader
batch_size = 192  # Set batch size for training
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
# Check the first few batches
for i, (images, _) in enumerate(dataloader):
    print(f"Loaded batch {i + 1} with size: {images.size()}")
    if i >= 2:  # Limit to a few batches
        break


Loaded batch 1 with size: torch.Size([192, 1, 64, 64])
Loaded batch 2 with size: torch.Size([192, 1, 64, 64])
Loaded batch 3 with size: torch.Size([192, 1, 64, 64])


In [ ]:
# Check the length of the dataset
print(f"Number of images in dataset: {len(dataset)}")

# Attempt to load a few images directly
for i in range(5):  # Try to load the first 5 images
    image, label = dataset[i]
    print(f"Image {i} shape: {image.shape}, label: {label}")


Number of images in dataset: 389765
Image 0 shape: torch.Size([1, 64, 64]), label: 0
Image 1 shape: torch.Size([1, 64, 64]), label: 0
Image 2 shape: torch.Size([1, 64, 64]), label: 0
Image 3 shape: torch.Size([1, 64, 64]), label: 0
Image 4 shape: torch.Size([1, 64, 64]), label: 0


In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4096 + 26, 2048),  # Input latent vector + 26 for one-hot letter encoding
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 64 * 64),  # Output flattened image of size 4096
            nn.Tanh()  # Normalize to [-1, 1]
        )

    def forward(self, noise, label):
        # Concatenate the noise vector and one-hot encoded label
        x = torch.cat((noise, label), dim=1)
        x = self.model(x)
        x = x.view(-1, 1, 64, 64)  # Reshape to (batch_size, 1 channel, 64x64)
        return x


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(64 * 64 + 26, 1024),  # Flattened image + one-hot letter encoding
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),  # Real/fake output
            nn.Sigmoid()
        )

    def forward(self, img, label):
        # Flatten the image and concatenate with the label
        img = img.view(img.size(0), -1)
        x = torch.cat((img, label), dim=1)
        x = self.model(x)
        return x


In [ ]:
import torch

def one_hot_encode(labels, num_classes):
    """
    Convert labels to one-hot encoded vectors.
    Args:
        labels: A tensor of labels (integers).
        num_classes: The number of classes (e.g., 26 for the alphabet).
    Returns:
        One-hot encoded labels of shape (batch_size, num_classes).
    """
    return torch.nn.functional.one_hot(labels, num_classes=num_classes).float()


In [ ]:
import torch
import pickle

# Function to save the model, optimizer, and epoch
def save_checkpoint(generator, discriminator, optimizer_G, optimizer_D, epoch, filename='checkpoint.pth'):
    checkpoint = {
        'generator_state_dict': generator.state_dict(),
        'discriminator_state_dict': discriminator.state_dict(),
        'optimizer_G_state_dict': optimizer_G.state_dict(),
        'optimizer_D_state_dict': optimizer_D.state_dict(),
        'epoch': epoch
    }
    with open(filename, 'wb') as f:
        pickle.dump(checkpoint, f)
    print(f"Checkpoint saved at epoch {epoch}")


In [ ]:
# Function to load the model, optimizer, and epoch
def load_checkpoint(generator, discriminator, optimizer_G, optimizer_D, filename='checkpoint.pth'):
    with open(filename, 'rb') as f:
        checkpoint = pickle.load(f)
        generator.load_state_dict(checkpoint['generator_state_dict'])
        discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
        optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
        optimizer_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
        start_epoch = checkpoint['epoch']
        print(f"Checkpoint loaded, resuming from epoch {start_epoch}")
    return start_epoch


In [ ]:
# Instantiate generator and discriminator
generator = Generator().cuda()       # If you're using GPU (add .cuda())
discriminator = Discriminator().cuda()  # Same for the discriminator

# Define the loss function
criterion = nn.BCELoss()  # Binary cross-entropy loss

# Define optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training loop
num_epochs = 100  # Set number of epochs for training

start_epoch = 0  # Default start epoch

# Load checkpoint if resuming
try:
    start_epoch = load_checkpoint(generator, discriminator, optimizer_G, optimizer_D, 'checkpoint.pth')
except FileNotFoundError:
    print("No checkpoint found, starting from scratch.")

for epoch in range(start_epoch, num_epochs):
    print(f"Starting epoch {epoch + 1}/{num_epochs}")

    for i, (real_images, labels) in enumerate(dataloader):
        real_images = real_images.cuda()
        labels = labels.cuda()

        batch_size = real_images.size(0)

        # One-hot encode the labels (if necessary)
        labels_one_hot = one_hot_encode(labels, num_classes=26).cuda()

        # Train Discriminator
        optimizer_D.zero_grad()

        # Train on real images
        real_labels = torch.ones(batch_size, 1).cuda()
        real_loss = criterion(discriminator(real_images, labels_one_hot), real_labels)

        # Generate fake images
        noise = torch.randn(batch_size, 4096).cuda()
        fake_images = generator(noise, labels_one_hot)

        # Train on fake images
        fake_labels = torch.zeros(batch_size, 1).cuda()
        fake_loss = criterion(discriminator(fake_images.detach(), labels_one_hot), fake_labels)

        # Total discriminator loss
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()

        # Generator loss (attempting to fool the discriminator)
        g_loss = criterion(discriminator(fake_images, labels_one_hot), real_labels)
        g_loss.backward()
        optimizer_G.step()

    save_checkpoint(generator, discriminator, optimizer_G, optimizer_D, epoch, filename='checkpoint.pth')

    # Print loss after every epoch
    print(f"Epoch [{epoch + 1}/{num_epochs}]  Loss D: {d_loss.item()}, Loss G: {g_loss.item()}")



Checkpoint loaded, resuming from epoch 57
Starting epoch 58/100
Checkpoint saved at epoch 57
Epoch [58/100]  Loss D: 0.29371801018714905, Loss G: 16.26841163635254
Starting epoch 59/100
Checkpoint saved at epoch 58
Epoch [59/100]  Loss D: 0.391812264919281, Loss G: 4.614320278167725
Starting epoch 60/100
Checkpoint saved at epoch 59
Epoch [60/100]  Loss D: 0.13217365741729736, Loss G: 7.454075813293457
Starting epoch 61/100
Checkpoint saved at epoch 60
Epoch [61/100]  Loss D: 0.19985803961753845, Loss G: 4.3376383781433105
Starting epoch 62/100
Checkpoint saved at epoch 61
Epoch [62/100]  Loss D: 0.2496483027935028, Loss G: 13.598889350891113
Starting epoch 63/100
Checkpoint saved at epoch 62
Epoch [63/100]  Loss D: 0.2742679715156555, Loss G: 10.855316162109375
Starting epoch 64/100
Checkpoint saved at epoch 63
Epoch [64/100]  Loss D: 0.020071817561984062, Loss G: 5.376232147216797
Starting epoch 65/100
Checkpoint saved at epoch 64
Epoch [65/100]  Loss D: 0.09332498162984848, Loss G: 

In [ ]:
import matplotlib.pyplot as plt

def generate_specific_letter(letter):
    # Map the letter to an integer label (A=0, B=1, ..., Z=25)
    letter_idx = ord(letter.upper()) - ord('A')

    # Create a one-hot encoding for the letter
    label = one_hot_encode(torch.tensor([letter_idx]), num_classes=26).cuda()

    # Generate random noise
    noise = torch.randn(1, 4096).cuda()

    # Generate the image
    generated_image = generator(noise, label)

    # Display the generated image
    plt.imshow(generated_image.squeeze().cpu().detach().numpy(), cmap='gray')
    plt.axis('off')
    plt.show()

# Generate the letter 'A'
generate_specific_letter('o')
